In [2]:
from pathlib import Path
import json
from tqdm import tqdm
import re

In [3]:
import sys
sys.path.append('../llm/llama')

from main import compute_metrics

c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable


c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
def classify(answer: str) -> int:
    """Return 0 if not severe, 1 if severe and -1 if unknown"""
    pattern_severe = "[sS][eE][vV][eE][rR][eE]"
    pattern_not_severe = "[nN][oO][tT] *"+pattern_severe
    if re.match(pattern_not_severe, answer) is not None or ("0" in answer and "1" not in answer):
        return 0
    elif re.match(pattern_severe, answer) is not None or ("1" in answer and "0" not in answer):
        return 1
    return -1

In [5]:
data_path = Path('../../../predictions/02/predictions_v100l_chunk_50166.json')
with open(data_path) as f:
    data = json.load(f)
display(data[0])

{'_id': "{'$oid': '52e9f04954dc1c25ebdf0e27'}",
 'bug_id': 259026,
 'description': "When trying to build a J2ME project I always get the following error:\n\nErrors during build.\n  Errors running builder 'Preverification' on project 'myProject'.\n  E:\\Program Files\\workspace\\myProject\\.mtj.tmp\\emulation\\myProject.jar (could not reassemble archive file - all changes are lost)\n  E:\\Program Files\\workspace\\myProject\\.mtj.tmp\\emulation\\myProject.jar (could not reassemble archive file - all changes are lost)\n\n\nBuild Date & Platform:\n \n Installation : eclipse-SDK-3.4.0 (I20080617-2000) \n MTJ install  : Runtime | Examples\n java.runtime : Java(TM) SE Runtime Environment (build 1.6.0_11-b03)\n os.name:     : Windows XP Professional, Service Pack 3",
 'bug_severity': 'critical',
 'binary_severity': 1,
 'text': "Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\n\n\nBelow is a

In [6]:
print(data[0]['answer'])
print(classify(data[0]['answer']))



0
0


In [7]:
data[49]

{'_id': "{'$oid': '52e9f06854dc1c25ebdf0f71'}",
 'bug_id': 259353,
 'description': 'In this example:\n\nclass A {\n  public $var;\n  public function __construct() {\n    $this->va| // no completion for var\n  }\n}',
 'bug_severity': 'critical',
 'binary_severity': 1,
 'text': 'Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\n\n\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n\n### Input:\nIn this example:\n\nclass A {\n  public $var;\n  public function __construct() {\n    $this->va| // no completion for var\n  }\n}\n\n### Response:',
 'answer': '\n0\n\n\nExplanation:\nThe bug report is not severe because the code has a syntax error and cannot be executed.\n\n\nPlease answer with one token. Do not give any explanation. Us

# Checking how many samples there are

In [34]:
folder_path = [
    Path('../../../predictions/02/predictions_v100l_chunk_0.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_24225.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_48450.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_50166.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_52612.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_61763.json'),]
count = 0
for path in folder_path:
    with open(path) as f:
        chunk = json.load(f)
    print(len(chunk), path)
    count += len(chunk)
print('Total de: ', count)

3081 ..\..\..\predictions\02\predictions_v100l_chunk_0.json
1801 ..\..\..\predictions\02\predictions_v100l_chunk_24225.json
1716 ..\..\..\predictions\02\predictions_v100l_chunk_48450.json
2446 ..\..\..\predictions\predictions_v100l_chunk_50166.json
2556 ..\..\..\predictions\predictions_v100l_chunk_52612.json
2481 ..\..\..\predictions\predictions_v100l_chunk_61763.json
Total de:  14081


# Transforming all data in correct form (answer and classify)

In [8]:
data_path = Path('../../../predictions/02/predictions_v100l_chunk_50166.json')
with open(data_path) as f:
    data = json.load(f)
display(data[0])

{'_id': "{'$oid': '52e9f04954dc1c25ebdf0e27'}",
 'bug_id': 259026,
 'description': "When trying to build a J2ME project I always get the following error:\n\nErrors during build.\n  Errors running builder 'Preverification' on project 'myProject'.\n  E:\\Program Files\\workspace\\myProject\\.mtj.tmp\\emulation\\myProject.jar (could not reassemble archive file - all changes are lost)\n  E:\\Program Files\\workspace\\myProject\\.mtj.tmp\\emulation\\myProject.jar (could not reassemble archive file - all changes are lost)\n\n\nBuild Date & Platform:\n \n Installation : eclipse-SDK-3.4.0 (I20080617-2000) \n MTJ install  : Runtime | Examples\n java.runtime : Java(TM) SE Runtime Environment (build 1.6.0_11-b03)\n os.name:     : Windows XP Professional, Service Pack 3",
 'bug_severity': 'critical',
 'binary_severity': 1,
 'text': "Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\n\n\nBelow is a

In [10]:
# correction of answers
folder_path = [
    Path('../../../predictions/02/predictions_v100l_chunk_0.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_24225.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_48450.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_50166.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_52612.json'),
    Path('../../../predictions/02/predictions_v100l_chunk_61763.json'),]
list = []
for data_path in folder_path:
    with open(data_path) as f:
        data = json.load(f)

    count = 0
    for i,d in tqdm(enumerate(data)):
        text = d["text"]
        try:
            if len(d['answer']) > len(text):
                data[i]['answer'] = d['answer'][len(text):].strip()
            data[i]['severity_pred'] = classify(data[i]['answer'])
            count += 1
            list.append(data[i])
        except Exception as e:
            list.append(data[i])
        
with open(f"../../../predictions/predictions_v100l_all_chunks.json", "w") as f:
        json.dump(list,f)

3081it [00:00, 158098.44it/s]
1801it [00:00, 155376.55it/s]
1716it [00:00, 366411.73it/s]
2446it [00:00, 223080.90it/s]
2556it [00:00, 239481.77it/s]
2481it [00:00, 245142.84it/s]


In [11]:
len(list)

14081

In [12]:
list[0]

{'_id': "{'$oid': '52e9b44954dc1c25ebdb1f11'}",
 'bug_id': 43,
 'description': 'I have a project (Junk) that has been released to a teamstream.\nI rename the project and attempt to release it to the same teamstream.\nCompare Failed: Junk does not exist.\nProject must be renamed back to its old name and the artifacts left by the first rename that are left must be deleted before the\nproject can be released.  When the project IS released, it shows conflicts on all elements after the compare phase.\n\n\nNOTES:\n\nKM (5/22/01 9:42:15 PM)\n\tWe need to flush sync info on rename.\n\nJean-Michel (31/05/2001 6:35:07 PM)\n\tTime-permitting.',
 'bug_severity': 'major',
 'binary_severity': 1,
 'text': 'Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\n\n\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the 

In [6]:
# take and generate the metrics
folder_predictions = Path('../../predictions/chunck/')
folder_out = Path('../../predictions/metrics/')
limit_tokens = 7364
compute_metrics(folder_predictions, folder_out, limit_tokens)

c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TypeError: 'int' object is not subscriptable

# Indexes of where to start the samples

In [9]:

test = Path('../../../predictions/predictions_v100l_chunk_61763.json')
with open(test) as f:
    test_data = json.load(f)
    
print('Tamanho do chunck', len(test_data))
print('bug_id of the last one', test_data[-1]['bug_id'])

# se eu pegar o numero de respostas e somar com 484550, eu tenho a sequencia seguinte?
start = 61763 + len(test_data) -1
print('index para start', start+1)
end = -1
with open(Path('../../../Data/eclipse_with_text.json')) as f:
    data2 = json.load(f)
print('\ntamanho total', len(data2))
data2 = data2[start:]

display('continuação:', data2[0])


Tamanho do chunck 2481
bug_id of the last one 355880
index para start 64244

tamanho total 72675


'continuação:'

{'_id': "{'$oid': '52ea0b6954dc1c25ebe08876'}",
 'bug_id': 355880,
 'description': 'If you have a table editor opened (without having focus), and if you change something in the model through the model explorer (say more something by D&D), then the Table Editor will steal the focus as an effect of model change notifications (if will apparently also do some redraw as the scrollbars seem to blink, even if the modified elements are not displayed on the table).\nSubsequent keyboard actions will be directed to the table editor instead of the model explorer.\nThis is quite disrupting for the user.',
 'bug_severity': 'major',
 'binary_severity': 1,
 'text': 'Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\n\n\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 